# Dvouvrstvý perceptron s využitím autogradu

- Úkolem cvičení je natrénovat opět **dvouvrstvý perceptron** pro klasifikaci obrázků na datasetu CIFAR-10, **tentokrát ovšem za použití modulu `ans.autograd` z minulého cvičení**.
- Vytvoříme novou třídu `ans.classification.TwoLayerPerceptronAutograd`, která bude obsahovat pouze dopředný průchod. Zpětný průchod zajistí automaticky implementace `ans.autograd.Variable.backprop` implementovaná v minulém cvičení.
- Vytvoříme rovněž nový modul `ans.nn`, do kterého budeme postupně přidávat funkcionalitu související s neuronovými sítěmi, abychom nemuseli neustále opakovat shodné bloky kódu.

**Poznámka**
- Tento notebook nezapíná [autoreload extension](https://ipython.org/ipython-doc/3/config/extensions/autoreload.html), protože testování využívá type checking pomocí `isinstance` a [to při reloadu modulu selhává](https://github.com/ipython/ipython/issues/12399).
- Při každé modifikaci modulu `ans.autograd` je proto bohužel nutné notebook restartovat a spustit celý kód znovu (např. "Run All") 😟

In [1]:
import sys
sys.path.append('..')  # import tests

import numpy as np
import matplotlib.pyplot as plt
import PIL
import torch
import torchvision
import ans
from tests import test_perceptron_autograd

# Příprava diferencovatelných operací

- Z minulého cvičení ma `Variable` implementované pouze operace `+`, `-`, `*` a `/`.
- Pro vícevrstvý perceptron (viz cvičení [two_layer_perceptron](two_layer_perceptron.ipynb)) optimalizovaný softmax křížovou entropií budeme ještě potřebovat alespoň
  - maticové násobení `@` pro vnitřní vrstvy perceptronu,
  - nelinearitu `sigmoid`,
  - `log`, `sum`, `exp` a `__getitem__` pro výpočet lossu.

## Maticové násobení `z = x @ y`

- Budeme uvažovat pouze minimálně dvourozměrné tensory, tj. matice x matice a výše.

**Dopředný průchod**

$$\boldsymbol{z} = \boldsymbol{x} \cdot \boldsymbol{y}$$

- $\boldsymbol{x} = [x_{\ldots,n,d}]$ je nejméně dvourozměrný tensor s rozměry $\ldots \times N \times D$,
- $\boldsymbol{y} = [y_{\ldots,d,k}]$ je nejméně dvourozměrný tensor s rozměry $\ldots \times D \times K$,
- $\boldsymbol{z} = [z_{\ldots,n,k}]$ je nejméně dvourozměrný tensor s rozměry $\ldots \times N \times K$,

**Zpětný průchod**

$$
\begin{align*}
    \overline{\boldsymbol{x}} &= \overline{\boldsymbol{z}} \cdot \boldsymbol{y}^\top \\
    \overline{\boldsymbol{y}} &= \boldsymbol{x}^\top \cdot \overline{\boldsymbol{z}}
\end{align*}
$$

- transpozicí $\boldsymbol{x}^\top$ pro vícerozměrné tensory se myslí prohození posledních dvou dimenzí $\ldots \times N \times D \rightarrow \ldots \times D \times N$

**Bonus**

- Zobecněte tak aby operátor `x @ y` fungoval i pro vektor (1D tensor) x matice a všechny ostatní kombinace podporované v PyTorchi.

### TODO: implementujte operátory [`ans.autograd.Variable.__matmul__`](../ans/autograd.py) a [`ans.autograd.Variable.__rmatmul__`](../ans/autograd.py)

In [ ]:
test_perceptron_autograd.TestMatMul.eval()

In [ ]:
test_perceptron_autograd.TestMatMulGeneral.eval()

## Sigmoid `z = x.sigmoid()`

**Dopředný průchod**

$$z = \frac{1}{1 + e^{-x}}$$
- $x$ je reálné číslo (skalár)
- $z$ je reálně číslo (skalár)

**Zpětný průchod**

Samostatně jako cvičení 😉

### TODO: implementujte metodu [`ans.autograd.Variable.sigmoid`](../ans/autograd.py)

In [ ]:
test_perceptron_autograd.TestSigmoid.eval()

## Umocňování `z = x.exp()`

**Dopředný průchod**

$$
z = e^{x}
$$
- $x$ je reálné číslo (skalár)
- $z$ je reálně číslo (skalár)

**Zpětný průchod**

Samostatně jako cvičení 😉

### TODO: implementujte metodu [`ans.autograd.Variable.exp`](../ans/autograd.py)

In [ ]:
test_perceptron_autograd.TestExp.eval()

## Přirozený logaritmus `z = x.log()`

- Nutné pro výpočet křížové entropie, kde se loss počítá jako logaritmus z pravděpodobnosti predikované pro target třídu.

**Dopředný průchod**

$$
z = \ln x
$$
- $x$ je reálné číslo (skalár)
- $z$ je reálně číslo (skalár)

**Zpětný průchod**

Samostatně jako cvičení 😉

### TODO: implementujte metodu [`ans.autograd.Variable.log`](../ans/autograd.py)

In [ ]:
test_perceptron_autograd.TestLog.eval()

## Výběr prvku `z = x[indices]`

- Nutné pro výpočet křížové entropie, kde se loss počítá jako $l = \ln \widehat{p}_y$, kde $\widehat{p}_y$ je pravděpodobnost predikovaná modelem pro target třídu $y$.

**Dopředný průchod**

- Operace je implementovaná v PyTorch jako `z = x[ids]`, kde `ids` může být `int`, `tuple[int,...]`, `list[int,...]` nebo `torch.Tensor`.
- Operaci si zjednodušíme a výběr omezíme na podmnožinu prvků `x`, kde se navíc nemůže opakovat výběr stejného prvku.
- Příklad:
  ``` python
  x = torch.tensor([0.5, 0.6, 0.7])
  z = x[[0]]  # z = torch.tensor([0.5]) ... ok
  z = x[[0, 1]]  # z = torch.tensor([0.5, 0.6]) ... ok
  z = x[[0, 0]]  # z = torch.tensor([0.5, 0.5]) ... validni v PyTorch, ale zde pro jednoduchost neuvazujeme (opakuje se prvek 0)
  z = x[[False, True, True]]  # z = torch.tensor([0.6, 0.7]) ... ok
  z = x[:2]  # z = torch.tensor([0.5, 0.6]) ... ok
  ...
  ```

**Zpětný průchod**

- Lokální gradient na *vybrané* prvky je jedna.
- Lokální gradient na *nevybrané* prvky je nula.

In [ ]:
test_perceptron_autograd.TestGetItem.eval()

## Součet prvků `z = x.sum(dim=..., keepdim=...)`

- Nutné pro výpočet softmaxu, kdy potřebujeme součet přes pravděpodobnosti tříd.

**Dopředný průchod**

- Operace je implementovaná v PyTorch jako `z = x.sum(...)`
- Podporované musí být oba parametry `dim` a `keepdim`, viz <https://pytorch.org/docs/stable/generated/torch.sum.html>

**Zpětný průchod**

- Lokální gradient na všechny prvky je jedna.

### TODO: implementujte metodu [`ans.autograd.Variable.sum`](../ans/autograd.py)

In [ ]:
test_perceptron_autograd.TestSum.eval()

## (Bonus) Průměr prvků `z = x.mean(dim=..., keepdim=...)`

- Podobné jako `x.sum(...)`, pouze s průměrem namísto součtu.

### TODO: implementujte metodu [`ans.autograd.Variable.mean`](../ans/autograd.py)

In [ ]:
test_perceptron_autograd.TestMean.eval()

# Třída `TwoLayerPerceptronAutograd`

- Cílem cvičení je natrénovat model dvouvrstvého perceptronu bez explicitního kódování zpětného průchodu, tj. s využitím autogradu.
- Model bude imeplementovat třída `ans.classification.TwoLayerPerceptronAutograd`.
- Parametry klasifikátoru budou reprezentovány jako `ans.autograd.Variable`, viz následující tabulku  
  | atribut   | typ                     | značení                                                            | rozměr       |
  |-----------|-------------------------|--------------------------------------------------------------------|--------------|
  | `weight1` | `ans.autograd.Variable` | $\boldsymbol{w}^{(1)} = \left[w_{d,h}^{(1)}\right]$                | $D \times H$ |
  | `bias1`   | `ans.autograd.Variable` | $\boldsymbol{b}^{(1)} = \left[b_1^{(1)}, \ldots, b_H^{(1)}\right]$ | $H$          |
  | `weight2` | `ans.autograd.Variable` | $\boldsymbol{w}^{(2)} = \left[w_{d,k}^{(2)}\right]$                | $H \times K$ |
  | `bias2`   | `ans.autograd.Variable` | $\boldsymbol{b}^{(2)} = \left[b_1^{(2)}, \ldots, b_K^{(2)}\right]$ | $K$          |
- Třída bude obsahovat pouze kód pro dopředný průchod (průchod sítí a výpočet lossu).
- Zpětný průchod bude volán jako
  ``` python
  # vynulovani gradientu
  ...
  # zpetna propagace
  loss.backprop()
  ```
- Po jeho provedení budou vyplněny atributy `.grad` všechny parametrů modelu, což umožní update pomocí SGD.
- **Důležité je před každým zpětným průchodem vynulovat gradienty parametrů, aby nedocházelo k jejich akumulaci přes různé dávky (minibatche)**, viz např. <https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch>.

**Funkce `softmax_cross_entropy`**
- Výpočet lossu implementujte jako funkci `ans.classification.softmax_cross_entropy`, aby se nemusely opakovat stejné kusy kódu v `TwoLayerPerceptronAutograd.train_step` a `TwoLayerPerceptronAutograd.val_Step`.
- Funkce by měla být agnostická vůči typu `logits` parametru:
  - pokud je `torch.Tensor`, výsledný loss bude rovněž `torch.Tensor`,
  - pokud je `Variable`, výsledný loss bude rovněž `Variable`.

### TODO: implementuje funkci [`ans.classification.softmax_cross_entropy`](../ans/classification.py).

In [ ]:
test_perceptron_autograd.TestSoftmaxCrossEntropy.eval()

### TODO: implementuje metodu [`ans.classification.TwoLayerPerceptronAutograd.__init__`](../ans/classification.py).

In [ ]:
test_perceptron_autograd.TestInit.eval()

### TODO: implementuje funkci [`ans.classification.TwoLayerPerceptronAutograd.train_step`](../ans/classification.py).

In [ ]:
test_perceptron_autograd.TestTrainStep.eval()

### TODO: implementuje funkci [`ans.classification.TwoLayerPerceptronAutograd.val_step`](../ans/classification.py).

In [ ]:
test_perceptron_autograd.TestValStep.eval()

## Preprocessing dat

- Pro trénování bude potřeba opět ještě preprocessing dat, podobně jako v minulých cvičeních.
- Použijeme shodný preprocessing jako ve cvičeních [linear_classification](linear_classification.ipynb) a [two_two_layer_perceptron](two_layer_perceptron.ipynb), tj. zploštení obrázků do vektorů a převod hodnot pixelů 0..255 do rozsahu 0..1 prostým vydělením 255.

### TODO: implementuje funkci `preprocess`.

In [14]:
def preprocess(img: PIL.Image) -> torch.Tensor:
    """
    Args:
        img: image of type PIL.Image
    Returns:
        x: 1-dimensional tensor; shape (num_pixels * num_channels,), dtype float32
    """
    
    ########################################
    # TODO: implement
    
    raise NotImplementedError
    
    # ENDTODO
    ########################################
    
    return x

In [ ]:
test_perceptron_autograd.TestPreprocess.eval(preprocess_fn=preprocess)

# Trénování klasifikátoru

- Pokud inicializujete parametry ve stejném pořadí a shodným způsobem jako ve cvičení [two_two_layer_perceptron](two_layer_perceptron.ipynb) (a pouze "obalíte" do `Variable`), měli byste dostat naprosto stejná čísla včetně progressu lossu!
- Nejlepší model uložte jako
  ``` python
  model.save('../output/two_layer_perceptron_autograd_weights.pt')
  ```

### TODO: Natrénujte dvouvrstvý perceptron tak, aby dosáhl alespoň 45 % (bonusově 50 %) *validační* accuracy.

In [ ]:
ans.utils.seed_everything(0)

########################################
# TODO: implement

# ...
# model = ans.classification.TwoLayerPerceptronAutograd(...)
# ...

raise NotImplementedError

# ENDTODO
########################################

In [ ]:
test_perceptron_autograd.TestValAccuracy45.eval(preprocess_fn=preprocess)

In [ ]:
test_perceptron_autograd.TestValAccuracy50.eval(preprocess_fn=preprocess)